# Init

In [1]:
from skfem import *
import numpy as np
from utils import *
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time


tol = 1e-8
intorder = 6
solver_type = 'mgcg'
refine_time = 7
element_type = 'P1'
sigma = 5
penalty = False
epsilon = 1e-6
example = 'ex3'
save_path = 'log/' + example + '_' + element_type + '_' + ('pen' if penalty else 'nopen') + '_' +'{}'.format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

# Solving

In [ ]:
# solving 


print('=======Arguments=======')
print('penalty:\t{}'.format(penalty))
print('element_type:\t{}'.format(element_type))
print('solver_type:\t{}'.format(solver_type))
print('tol:\t{}'.format(tol))
print('intorder:\t{}'.format(intorder))
print('refine_time:\t{}'.format(refine_time))
print('epsilon:\t{}'.format(epsilon))
print('sigma:\t{}'.format(sigma))
print('=======Results=======')

time_start = time.time()

df_list = []

ep = epsilon
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, refine_time+1):

    m.refine()

    if penalty:
        uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)
    else:
        uh0, basis = solve_problem1(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)

    U = basis['u'].interpolate(uh0).value

    # compute errors

    L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
    Du = get_DuError(basis['u'], uh0)
    H1u = Du + L2u
    if penalty:
        D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
    else:
        D2u = get_D2uError(basis['u'], uh0)
    epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
    D2u_list.append(D2u)
    epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us

# store data
data = np.array([L2s, H1s, H2s, epus])
df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
df_list.append(df)

print('epsilon =', epsilon)
print('  h    L2u   H1u   H2u   epu')
for i in range(H2s.shape[0] - 1):
    print(
        '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
            -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
            -np.log2(H2s[i + 1] / H2s[i]),
            -np.log2(epus[i + 1] / epus[i])))
    print(
        '2^-' + str(i + 2), ' {:.3e}  {:.3e}  {:.3e}  {:.3e}'.format(
            L2s[i + 1], H1s[i + 1],
            H2s[i + 1],
            epus[i + 1]))

time_end = time.time()

# df.to_csv(save_path+'.csv')
print('Total Time Cost {:.2f} s'.format(time_end-time_start))

=======Arguments=======
penalty:	False
element_type:	P1
solver_type:	mgcg
tol:	1e-08
intorder:	6
refine_time:	7
epsilon:	1e-06
sigma:	5
=======Results=======
